# 1 - Software Efort Estimation Using Stacked Ensemble Technique and Hybrid Principal Component Regression and Multivariate Adaptive Regression Splines

Divisão: Cross_validation e kfold.

In [22]:
import os
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV, cross_val_predict
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils.validation import check_X_y
import numpy as np

# Function to load datasets
def load_datasets(directory):
    datasets = {}
    if not os.path.exists(directory):
        print(f"Directory '{directory}' not found.")
        return datasets

    for file in os.listdir(directory):
        filepath = os.path.join(directory, file)
        filename, ext = os.path.splitext(file)

        try:
            if ext == '.txt':
                with open(filepath, 'r') as f:
                    first_line = f.readline()
                    delimiter = ',' if ',' in first_line else ('\t' if '\t' in first_line else ' ')
                datasets[filename] = pd.read_csv(filepath, delimiter=delimiter)
                print(f"{filename}: Detected delimiter - '{delimiter}'")
            else:
                print(f"Unsupported format: {file}")
        except Exception as e:
            print(f"Error loading {file}: {e}")

    print(f"{len(datasets)} files successfully loaded from {directory}.")
    return datasets

# Function to process a single model
def process_model(X, y, model_name, model, param_grid, kf):
    metrics = {
        'rmse': [],
        'mse': [],
        'mae': [],
        'r2': [],
    }

    grid = GridSearchCV(model, param_grid[model_name], cv=kf, scoring='neg_mean_squared_error', error_score='raise')

    try:
        grid.fit(X, y)
        best_model = grid.best_estimator_

        # Predictions with cross-validation
        y_pred = cross_val_predict(best_model, X, y, cv=kf)

        # Calculate metrics
        metrics['rmse'].append(np.sqrt(mean_squared_error(y, y_pred)))
        metrics['mse'].append(mean_squared_error(y, y_pred))
        metrics['mae'].append(mean_absolute_error(y, y_pred))
        metrics['r2'].append(r2_score(y, y_pred))

        return {
            'rmse': np.mean(metrics['rmse']),
            'mse': np.mean(metrics['mse']),
            'mae': np.mean(metrics['mae']),
            'r2': np.mean(metrics['r2']),
            'best_params': grid.best_params_
        }

    except Exception as e:
        print(f"{model_name}: Training failure - {e}")
        return None

# Function to calculate average metrics
def calculate_average_metrics(aggregated_results):
    average_metrics = {}
    for dataset, models in aggregated_results.items():
        average_metrics[dataset] = {}
        for model_name, metrics in models.items():
            average_metrics[dataset][model_name] = {
                'rmse': np.mean(metrics['rmse']),
                'mse': np.mean(metrics['mse']),
                'mae': np.mean(metrics['mae']),
                'r2': np.mean(metrics['r2']),
            }
    return average_metrics

# Function to train and test regression models
def test_regression_models(data, output_directory, n_splits=5, num_runs=30):
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(),
    }

    param_grid = {
        'LinearRegression': {},
        'Ridge': {'alpha': [0.1, 1, 10]},
        'RandomForest': {'n_estimators': [50, 100, 200]},
    }

    aggregated_results = {}

    for name, df in data.items():
        df = df.dropna()

        if df.shape[0] < 2 or df.shape[1] < 2:
            print(f"{name}: Dataset too small for training. Size: {df.shape}")
            continue

        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]

        try:
            X, y = check_X_y(X, y)
        except ValueError as e:
            print(f"{name}: Data validation error - {e}")
            continue

        aggregated_results[name] = {model_name: {
            'rmse': [], 'mse': [], 'mae': [], 'r2': []
        } for model_name in models.keys()}

        for run in range(num_runs):
            kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

            for model_name, model in models.items():
                metrics = process_model(X, y, model_name, model, param_grid, kf)
                if metrics:
                    for metric, value in metrics.items():
                        if metric != 'best_params':
                            aggregated_results[name][model_name][metric].append(value)

        # Calculate averages
        for model_name in models.keys():
            for metric in ['rmse', 'mse', 'mae', 'r2']:
                aggregated_results[name][model_name][metric] = np.mean(aggregated_results[name][model_name][metric])

    # Calculate average metrics
    average_metrics = calculate_average_metrics(aggregated_results)

    # Save results to Excel
    results_df = pd.DataFrame.from_dict({
        f'{dataset}_{model}': metrics
        for dataset, models in aggregated_results.items()
        for model, metrics in models.items()
    }, orient='index')

    output_path = os.path.join(output_directory, 'artigo1-abordagem2-teste-30runs.xlsx')

    try:
        os.makedirs(output_directory, exist_ok=True)
        results_df.to_excel(output_path)
        print(f"Results saved at '{output_path}'.")
    except Exception as e:
        print(f"Error saving results: {e}")

    # Print average metrics
    print("\nAverage Metrics:")
    for dataset, models in average_metrics.items():
        print(f"\nDataset: {dataset}")
        for model_name, metrics in models.items():
            print(f"Model: {model_name}")
            for metric, value in metrics.items():
                print(f"{metric}: {value:.4f}")

# Process multiple directories
output_directory = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\artigo1'

directories = [
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\1-tratamento',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\2-geracao\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\2-geracao-variaveis\MIMIC-25',
]

for directory in directories:
    data = load_datasets(directory)
    if data:
        test_regression_models(data, output_directory)

Unsupported format: resultado_consolidado_tratamento.xlsx
tratamento_china: Detected delimiter - ','
Unsupported format: tratamento_china_resultados_kfold.xlsx
tratamento_cocomo81: Detected delimiter - ','
Unsupported format: tratamento_cocomo81_resultados_kfold.xlsx
tratamento_desharnais: Detected delimiter - ','
Unsupported format: tratamento_desharnais_resultados_kfold.xlsx
tratamento_maxwell: Detected delimiter - ','
Unsupported format: tratamento_maxwell_resultados_kfold.xlsx
4 files successfully loaded from C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado.
Results saved at 'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\artigo1\artigo1-abordagem2-teste-30runs.xlsx'.

Average Metrics:

Dataset: tratamento_china
Model: LinearRegression
rmse: 0.0196
mse: 0.0004
mae: 0.0069
r2: 0.9728
Model: Ridge
rmse: 0.0179
mse: 0.0003
mae

# 2 - Estimating Software Development Efforts Using a Random Forest-Based Stacked Ensemble Approach

Divisão: cross_validation e kfold. 

In [13]:
import os
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV, cross_val_predict
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils.validation import check_X_y
from joblib import Parallel, delayed
import numpy as np

# Função para carregar datasets
def load_datasets(directory):
    datasets = {}
    if not os.path.exists(directory):
        print(f"Diretório '{directory}' não encontrado.")
        return datasets

    for file in os.listdir(directory):
        filepath = os.path.join(directory, file)
        filename, ext = os.path.splitext(file)

        try:
            if ext == '.txt':
                with open(filepath, 'r') as f:
                    first_line = f.readline()
                    delimiter = ',' if ',' in first_line else ('\t' if '\t' in first_line else ' ')
                datasets[filename] = pd.read_csv(filepath, delimiter=delimiter)
                print(f"{filename}: Delimitador detectado - '{delimiter}'")
            else:
                print(f"Formato não suportado: {file}")
        except Exception as e:
            print(f"Erro ao carregar {file}: {e}")

    print(f"{len(datasets)} arquivos carregados com sucesso de {directory}.")
    return datasets

# Função para processar um único modelo
def process_model(X, y, model_name, model, param_grid, kf):
    metrics = {
        'mae': [],
        'rmse': [],
        'r2': []
    }

    grid = GridSearchCV(model, param_grid[model_name], cv=kf, scoring='neg_mean_squared_error', error_score='raise')

    try:
        grid.fit(X, y)
        best_model = grid.best_estimator_

        # Previsões com validação cruzada
        y_pred = cross_val_predict(best_model, X, y, cv=kf)

        # Cálculo das métricas
        metrics['mae'].append(mean_absolute_error(y, y_pred))
        metrics['rmse'].append(np.sqrt(mean_squared_error(y, y_pred)))
        metrics['r2'].append(r2_score(y, y_pred))

        return {
            'mae': np.mean(metrics['mae']),
            'rmse': np.mean(metrics['rmse']),
            'r2': np.mean(metrics['r2']),
            'best_params': grid.best_params_
        }

    except Exception as e:
        print(f"{model_name}: Falha no treinamento - {e}")
        return None

# Função para treinar e testar modelos de regressão
def test_regression_models(data, output_directory, n_splits=5, n_runs=30):
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        # 'Lasso': Lasso(),
        'RandomForest': RandomForestRegressor(),
        # 'GradientBoosting': GradientBoostingRegressor()
    }

    param_grid = {
        'LinearRegression': {},
        'Ridge': {'alpha': [0.1, 1, 10]},
        # 'Lasso': {'alpha': [0.1, 0.5, 1]},
        'RandomForest': {'n_estimators': [50, 100, 200]},
        # 'GradientBoosting': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]}
    }

    results = {}

    for name, df in data.items():
        df = df.dropna()

        if df.shape[0] < 2 or df.shape[1] < 2:
            print(f"{name}: Dataset insuficiente para treino. Tamanho: {df.shape}")
            continue

        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]

        try:
            X, y = check_X_y(X, y)
        except ValueError as e:
            print(f"{name}: Erro de validação dos dados - {e}")
            continue

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=None)

        # Processar modelos em paralelo
        model_results = Parallel(n_jobs=-1)(
            delayed(process_model)(X, y, model_name, model, param_grid, kf)
            for model_name, model in models.items()
        )

        for model_name, metrics in zip(models.keys(), model_results):
            if metrics:
                results[f'{name}_{model_name}'] = metrics
                print(f"{name} - {model_name}: MAE = {metrics['mae']:.4f}, RMSE = {metrics['rmse']:.4f}, R² = {metrics['r2']:.4f}")

    results_df = pd.DataFrame.from_dict(results, orient='index')
    output_path = os.path.join(output_directory, 'artigo2-abordagem2-teste.xlsx')

    try:
        os.makedirs(output_directory, exist_ok=True)
        results_df.to_excel(output_path)
        print(f"Resultados salvos em '{output_path}'.")
    except Exception as e:
        print(f"Erro ao salvar os resultados: {e}")

# Processamento de múltiplos diretórios
output_directory = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\artigo2' 

directories = [
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\1-tratamento',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\2-geracao\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\2-geracao-variaveis\MIMIC-25',
]

for directory in directories:
    data = load_datasets(directory)
    if data:
        test_regression_models(data, output_directory)


Formato não suportado: resultado_consolidado_tratamento.xlsx
tratamento_china: Delimitador detectado - ','
Formato não suportado: tratamento_china_resultados_kfold.xlsx
tratamento_cocomo81: Delimitador detectado - ','
Formato não suportado: tratamento_cocomo81_resultados_kfold.xlsx
tratamento_desharnais: Delimitador detectado - ','
Formato não suportado: tratamento_desharnais_resultados_kfold.xlsx
tratamento_maxwell: Delimitador detectado - ','
Formato não suportado: tratamento_maxwell_resultados_kfold.xlsx
4 arquivos carregados com sucesso de C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado.
tratamento_china - LinearRegression: MAE = 0.0071, RMSE = 0.0201, R² = 0.9712
tratamento_china - Ridge: MAE = 0.0065, RMSE = 0.0176, R² = 0.9780
tratamento_china - RandomForest: MAE = 0.0085, RMSE = 0.0284, R² = 0.9426
tratamento_cocomo81 - LinearRegression: MAE = 0.0883, RMSE = 0.1445, R² = 0.1698
trat

In [15]:
import os
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV, cross_val_predict
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils.validation import check_X_y
from joblib import Parallel, delayed
import numpy as np

# Função para carregar datasets
def load_datasets(directory):
    datasets = {}
    if not os.path.exists(directory):
        print(f"Diretório '{directory}' não encontrado.")
        return datasets

    for file in os.listdir(directory):
        filepath = os.path.join(directory, file)
        filename, ext = os.path.splitext(file)

        try:
            if ext == '.txt':
                with open(filepath, 'r') as f:
                    first_line = f.readline()
                    delimiter = ',' if ',' in first_line else ('\t' if '\t' in first_line else ' ')
                datasets[filename] = pd.read_csv(filepath, delimiter=delimiter)
                print(f"{filename}: Delimitador detectado - '{delimiter}'")
            else:
                print(f"Formato não suportado: {file}")
        except Exception as e:
            print(f"Erro ao carregar {file}: {e}")

    print(f"{len(datasets)} arquivos carregados com sucesso de {directory}.")
    return datasets

# Função para processar um único modelo com múltiplas execuções
def process_model(X, y, model_name, model, param_grid, kf, n_runs):
    metrics = {
        'mae': [],
        'rmse': [],
        'r2': []
    }

    for _ in range(n_runs):
        grid = GridSearchCV(model, param_grid[model_name], cv=kf, scoring='neg_mean_squared_error', error_score='raise')
        
        try:
            grid.fit(X, y)
            best_model = grid.best_estimator_

            # Previsões com validação cruzada
            y_pred = cross_val_predict(best_model, X, y, cv=kf)

            # Cálculo das métricas
            metrics['mae'].append(mean_absolute_error(y, y_pred))
            metrics['rmse'].append(np.sqrt(mean_squared_error(y, y_pred)))
            metrics['r2'].append(r2_score(y, y_pred))
        except Exception as e:
            print(f"{model_name}: Falha no treinamento - {e}")
            continue

    # Retornar a média das métricas ao longo de todas as execuções
    return {
        'mae': np.mean(metrics['mae']),
        'rmse': np.mean(metrics['rmse']),
        'r2': np.mean(metrics['r2']),
        'best_params': grid.best_params_ if metrics['mae'] else None
    }

# Função para treinar e testar modelos de regressão
def test_regression_models(data, output_directory, n_splits=5, n_runs=30):
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(),
    }

    param_grid = {
        'LinearRegression': {},
        'Ridge': {'alpha': [0.1, 1, 10]},
        'RandomForest': {'n_estimators': [50, 100, 200]},
    }

    results = {}

    for name, df in data.items():
        df = df.dropna()

        if df.shape[0] < 2 or df.shape[1] < 2:
            print(f"{name}: Dataset insuficiente para treino. Tamanho: {df.shape}")
            continue

        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]

        try:
            X, y = check_X_y(X, y)
        except ValueError as e:
            print(f"{name}: Erro de validação dos dados - {e}")
            continue

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=None)

        # Processar modelos em paralelo
        model_results = Parallel(n_jobs=-1)(
            delayed(process_model)(X, y, model_name, model, param_grid, kf, n_runs)
            for model_name, model in models.items()
        )

        for model_name, metrics in zip(models.keys(), model_results):
            if metrics:
                results[f'{name}_{model_name}'] = metrics
                print(f"{name} - {model_name}: MAE = {metrics['mae']:.4f}, RMSE = {metrics['rmse']:.4f}, R² = {metrics['r2']:.4f}")

    results_df = pd.DataFrame.from_dict(results, orient='index')
    output_path = os.path.join(output_directory, 'resultados.xlsx')

    try:
        os.makedirs(output_directory, exist_ok=True)
        results_df.to_excel(output_path)
        print(f"Resultados salvos em '{output_path}'.")
    except Exception as e:
        print(f"Erro ao salvar os resultados: {e}")

# Função para processar múltiplos diretórios e salvar resultados
def process_multiple_directories(directories, base_output_directory, n_runs=30):
    for directory in directories:
        # Extrair o nome do diretório atual para organizar os resultados
        directory_name = os.path.basename(directory.rstrip(os.sep))
        output_subdirectory = os.path.join(base_output_directory, directory_name)

        # Carregar os datasets do diretório atual
        data = load_datasets(directory)

        # Se houver datasets carregados, processar os modelos
        if data:
            test_regression_models(data, output_subdirectory, n_runs=n_runs)

# Diretórios de entrada e saída
output_directory = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\artigo2'

directories = [
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\1-tratamento',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\2-geracao\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\2-geracao-variaveis\MIMIC-25',
]

# Processar os diretórios
process_multiple_directories(directories, output_directory, n_runs=30)


Formato não suportado: resultado_consolidado_tratamento.xlsx
tratamento_china: Delimitador detectado - ','
Formato não suportado: tratamento_china_resultados_kfold.xlsx
tratamento_cocomo81: Delimitador detectado - ','
Formato não suportado: tratamento_cocomo81_resultados_kfold.xlsx
tratamento_desharnais: Delimitador detectado - ','
Formato não suportado: tratamento_desharnais_resultados_kfold.xlsx
tratamento_maxwell: Delimitador detectado - ','
Formato não suportado: tratamento_maxwell_resultados_kfold.xlsx
4 arquivos carregados com sucesso de C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado.
tratamento_china - LinearRegression: MAE = 0.0070, RMSE = 0.0191, R² = 0.9739
tratamento_china - Ridge: MAE = 0.0067, RMSE = 0.0175, R² = 0.9782
tratamento_china - RandomForest: MAE = 0.0078, RMSE = 0.0263, R² = 0.9506
tratamento_cocomo81 - LinearRegression: MAE = 0.0915, RMSE = 0.1504, R² = 0.0954
trat

In [20]:
import os
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV, cross_val_predict
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils.validation import check_X_y
from joblib import Parallel, delayed
import numpy as np

# Função para carregar datasets
def load_datasets(directory):
    datasets = {}
    if not os.path.exists(directory):
        print(f"Diretório '{directory}' não encontrado.")
        return datasets

    for file in os.listdir(directory):
        filepath = os.path.join(directory, file)
        filename, ext = os.path.splitext(file)

        # Processar apenas arquivos com extensão .txt
        if ext != '.txt':
            print(f"Ignorando arquivo não suportado: {file}")
            continue

        try:
            # Ler a primeira linha para detectar o delimitador
            with open(filepath, 'r') as f:
                first_line = f.readline()
                delimiter = ',' if ',' in first_line else ('\t' if '\t' in first_line else ' ')

            # Tentar carregar o dataset
            df = pd.read_csv(filepath, delimiter=delimiter)
            if df.empty:
                print(f"{filename}: Arquivo está vazio ou inválido, ignorando.")
                continue

            datasets[filename] = df
            print(f"{filename}: Delimitador detectado - '{delimiter}'")

        except pd.errors.ParserError as e:
            print(f"{filename}: Erro ao processar arquivo - {e}")
        except Exception as e:
            print(f"Erro ao carregar {file}: {e}")

    print(f"{len(datasets)} arquivos carregados com sucesso de {directory}.")
    return datasets

# Função para processar um único modelo com múltiplas execuções
def process_model(X, y, model_name, model, param_grid, kf, n_runs):
    metrics = {
        'mae': [],
        'rmse': [],
        'r2': []
    }

    for _ in range(n_runs):
        grid = GridSearchCV(model, param_grid[model_name], cv=kf, scoring='neg_mean_squared_error', error_score='raise')

        try:
            grid.fit(X, y)
            best_model = grid.best_estimator_

            # Previsões com validação cruzada
            y_pred = cross_val_predict(best_model, X, y, cv=kf)

            # Cálculo das métricas
            metrics['mae'].append(mean_absolute_error(y, y_pred))
            metrics['rmse'].append(np.sqrt(mean_squared_error(y, y_pred)))
            metrics['r2'].append(r2_score(y, y_pred))
        except Exception as e:
            print(f"{model_name}: Falha no treinamento - {e}")
            continue

    # Retornar a média das métricas ao longo de todas as execuções
    return {
        'mae': np.mean(metrics['mae']),
        'rmse': np.mean(metrics['rmse']),
        'r2': np.mean(metrics['r2']),
        'best_params': grid.best_params_ if metrics['mae'] else None
    }

# Função para treinar e testar modelos de regressão
def test_regression_models(data, output_directory, n_splits=5, n_runs=30):
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(),
    }

    param_grid = {
        'LinearRegression': {},
        'Ridge': {'alpha': [0.1, 1, 10]},
        'RandomForest': {'n_estimators': [50, 100, 200]},
    }

    results = {}

    for name, df in data.items():
        # Garantir que o dataset não esteja vazio
        if df.empty or df.shape[0] < 2 or df.shape[1] < 2:
            print(f"{name}: Dataset insuficiente ou vazio. Ignorando.")
            continue

        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]

        try:
            X, y = check_X_y(X, y)
        except ValueError as e:
            print(f"{name}: Erro de validação dos dados - {e}")
            continue

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=None)

        # Processar modelos em paralelo
        model_results = Parallel(n_jobs=-1)(
            delayed(process_model)(X, y, model_name, model, param_grid, kf, n_runs)
            for model_name, model in models.items()
        )

        for model_name, metrics in zip(models.keys(), model_results):
            if metrics:
                results[f'{name}_{model_name}'] = metrics
                print(f"{name} - {model_name}: MAE = {metrics['mae']:.4f}, RMSE = {metrics['rmse']:.4f}, R² = {metrics['r2']:.4f}")

    results_df = pd.DataFrame.from_dict(results, orient='index')
    output_path = os.path.join(output_directory, 'resultados.xlsx')

    try:
        os.makedirs(output_directory, exist_ok=True)
        results_df.to_excel(output_path)
        print(f"Resultados salvos em '{output_path}'.")
    except Exception as e:
        print(f"Erro ao salvar os resultados: {e}")

# Função para processar múltiplos diretórios e salvar resultados
def process_multiple_directories(directories, base_output_directory, n_runs=30):
    for directory in directories:
        # Extrair o nome do diretório atual para organizar os resultados
        directory_name = os.path.basename(directory.rstrip(os.sep))
        output_subdirectory = os.path.join(base_output_directory, directory_name)

        # Carregar os datasets do diretório atual
        data = load_datasets(directory)

        # Se houver datasets carregados, processar os modelos
        if data:
            test_regression_models(data, output_subdirectory, n_runs=n_runs)

# Diretórios de entrada e saída
output_directory = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\artigo2'

directories = [
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\1-tratamento',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\2-geracao\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\2-geracao-variaveis\MIMIC-25',
]

# Processar os diretórios
process_multiple_directories(directories, output_directory, n_runs=30)


Ignorando arquivo não suportado: resultado_consolidado_tratamento.xlsx
tratamento_china: Delimitador detectado - ','
Ignorando arquivo não suportado: tratamento_china_resultados_kfold.xlsx
tratamento_cocomo81: Delimitador detectado - ','
Ignorando arquivo não suportado: tratamento_cocomo81_resultados_kfold.xlsx
tratamento_desharnais: Delimitador detectado - ','
Ignorando arquivo não suportado: tratamento_desharnais_resultados_kfold.xlsx
tratamento_maxwell: Delimitador detectado - ','
Ignorando arquivo não suportado: tratamento_maxwell_resultados_kfold.xlsx
4 arquivos carregados com sucesso de C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado.
tratamento_china - LinearRegression: MAE = 0.0071, RMSE = 0.0195, R² = 0.9729
tratamento_china - Ridge: MAE = 0.0066, RMSE = 0.0174, R² = 0.9784
tratamento_china - RandomForest: MAE = 0.0078, RMSE = 0.0264, R² = 0.9501
tratamento_cocomo81 - LinearRegress

# 3 - Software Effort Estimation using Ensemble Learning.

Divisão: 80 treino e 20 teste.

In [12]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.utils.validation import check_X_y
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np

# Função para calcular RAE
def calculate_rae(y_true, y_pred):
    return np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(y_true - np.mean(y_true)))

# Função para calcular RRSE
def calculate_rrse(y_true, y_pred):
    return np.sqrt(np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))

# Função para carregar datasets
def load_datasets(directory):
    datasets = {}
    if not os.path.exists(directory):
        print(f"Diretório '{directory}' não encontrado.")
        return datasets

    for file in os.listdir(directory):
        filepath = os.path.join(directory, file)
        filename, ext = os.path.splitext(file)

        try:
            if ext == '.txt':
                with open(filepath, 'r') as f:
                    first_line = f.readline()
                    delimiter = ',' if ',' in first_line else ('\t' if '\t' in first_line else ' ')
                datasets[filename] = pd.read_csv(filepath, delimiter=delimiter)
                print(f"{filename}: Delimitador detectado - '{delimiter}'")
            else:
                print(f"Formato não suportado: {file}")
        except Exception as e:
            print(f"Erro ao carregar {file}: {e}")

    print(f"{len(datasets)} arquivos carregados com sucesso de {directory}.")
    return datasets

# Função para processar um único modelo
def process_model(X_train, X_test, y_train, y_test, model_name, model, param_grid, repetitions):
    metrics = {'mae': [], 'rmse': [], 'rae': [], 'rrse': []}

    for _ in range(repetitions):
        grid = GridSearchCV(model, param_grid[model_name], cv=5, scoring='neg_mean_absolute_error', error_score='raise')

        try:
            grid.fit(X_train, y_train)
            best_model = grid.best_estimator_

            # Previsões no conjunto de teste
            y_pred = best_model.predict(X_test)

            # Cálculo das métricas
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            rae = calculate_rae(y_test, y_pred)
            rrse = calculate_rrse(y_test, y_pred)

            metrics['mae'].append(mae)
            metrics['rmse'].append(rmse)
            metrics['rae'].append(rae)
            metrics['rrse'].append(rrse)

        except Exception as e:
            print(f"{model_name}: Falha no treinamento - {e}")
            continue

    # Calcular médias das métricas
    return {
        'mae': np.mean(metrics['mae']),
        'rmse': np.mean(metrics['rmse']),
        'rae': np.mean(metrics['rae']),
        'rrse': np.mean(metrics['rrse']),
        'best_params': grid.best_params_ if 'grid' in locals() else {}
    }

# Função para treinar e testar modelos de regressão
def test_regression_models(data, directory, output_directory, repetitions=30):
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(),
    }

    param_grid = {
        'LinearRegression': {},
        'Ridge': {'alpha': [0.1, 1, 10]},
        'RandomForest': {'n_estimators': [50, 100, 200]},
    }

    results = {}

    for name, df in tqdm(data.items(), desc="Processando datasets"):
        df = df.dropna()

        if df.shape[0] < 2 or df.shape[1] < 2:
            print(f"{name}: Dataset insuficiente para treino. Tamanho: {df.shape}")
            continue

        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]

        try:
            X, y = check_X_y(X, y)
        except ValueError as e:
            print(f"{name}: Erro de validação dos dados - {e}")
            continue

        # Divisão dos dados em treinamento (80%) e teste (20%)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Processar cada modelo em paralelo
        model_results = Parallel(n_jobs=-1)(
            delayed(process_model)(X_train, X_test, y_train, y_test, model_name, model, param_grid, repetitions)
            for model_name, model in models.items()
        )

        # Armazenar resultados
        for model_name, metrics in zip(models.keys(), model_results):
            results[f'{name}_{model_name}'] = metrics
            print(f"{name} - {model_name}: MAE = {metrics['mae']:.4f}, RMSE = {metrics['rmse']:.4f}, "
                  f"RAE = {metrics['rae']:.4f}, RRSE = {metrics['rrse']:.4f}")

    results_df = pd.DataFrame.from_dict(results, orient='index')
    output_path = os.path.join(output_directory, f'{os.path.basename(directory)}_resultados.xlsx')

    try:
        os.makedirs(output_directory, exist_ok=True)
        results_df.to_excel(output_path)
        print(f"Resultados salvos em '{output_path}'.")
    except Exception as e:
        print(f"Erro ao salvar os resultados: {e}")

# Processamento de múltiplos diretórios
output_directory = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados'
directories = [
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\1-tratamento',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\2-geracao\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\2-geracao-variaveis\MIMIC-25',
]

for directory in tqdm(directories, desc="Processando diretórios"):
    data = load_datasets(directory)
    if data:
        test_regression_models(data, directory, output_directory, repetitions=30)

Processando diretórios:   0%|          | 0/6 [00:00<?, ?it/s]

Formato não suportado: resultado_consolidado_tratamento.xlsx
tratamento_china: Delimitador detectado - ','
Formato não suportado: tratamento_china_resultados_kfold.xlsx
tratamento_cocomo81: Delimitador detectado - ','
Formato não suportado: tratamento_cocomo81_resultados_kfold.xlsx
tratamento_desharnais: Delimitador detectado - ','
Formato não suportado: tratamento_desharnais_resultados_kfold.xlsx
tratamento_maxwell: Delimitador detectado - ','
Formato não suportado: tratamento_maxwell_resultados_kfold.xlsx
4 arquivos carregados com sucesso de C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado.


tratamento_china - LinearRegression: MAE = 0.0082, RMSE = 0.0233, RAE = 0.1685, RRSE = 0.2319
tratamento_china - Ridge: MAE = 0.0067, RMSE = 0.0158, RAE = 0.1371, RRSE = 0.1578
tratamento_china - RandomForest: MAE = 0.0070, RMSE = 0.0285, RAE = 0.1436, RRSE = 0.2836


tratamento_cocomo81 - LinearRegression: MAE = 0.1214, RMSE = 0.1587, RAE = 3.7213, RRSE = 3.2520
tratamento_cocomo81 - Ridge: MAE = 0.0488, RMSE = 0.0608, RAE = 1.4965, RRSE = 1.2460
tratamento_cocomo81 - RandomForest: MAE = 0.0237, RMSE = 0.0428, RAE = 0.7266, RRSE = 0.8763


tratamento_desharnais - LinearRegression: MAE = 0.0899, RMSE = 0.1102, RAE = 0.7854, RRSE = 0.7218
tratamento_desharnais - Ridge: MAE = 0.0800, RMSE = 0.1019, RAE = 0.6990, RRSE = 0.6677
tratamento_desharnais - RandomForest: MAE = 0.0757, RMSE = 0.0909, RAE = 0.6612, RRSE = 0.5953


...

tratamento_china - LinearRegression: MAE = 0.0010, RMSE = 0.0013, RAE = 0.0200, RRSE = 0.0131
tratamento_china - Ridge: MAE = 0.0031, RMSE = 0.0059, RAE = 0.0637, RRSE = 0.0585
tratamento_china - RandomForest: MAE = 0.0035, RMSE = 0.0211, RAE = 0.0711, RRSE = 0.2104


tratamento_cocomo81 - LinearRegression: MAE = 0.0709, RMSE = 0.0830, RAE = 2.1712, RRSE = 1.7004
tratamento_cocomo81 - Ridge: MAE = 0.0559, RMSE = 0.0616, RAE = 1.7126, RRSE = 1.2636
tratamento_cocomo81 - RandomForest: MAE = 0.0192, RMSE = 0.0330, RAE = 0.5874, RRSE = 0.6768


tratamento_desharnais - LinearRegression: MAE = 0.0862, RMSE = 0.1091, RAE = 0.7528, RRSE = 0.7148
tratamento_desharnais - Ridge: MAE = 0.0849, RMSE = 0.1084, RAE = 0.7410, RRSE = 0.7097
tratamento_desharnais - RandomForest: MAE = 0.0742, RMSE = 0.0932, RAE = 0.6477, RRSE = 0.6102


Processando diretórios: 100%|██████████| 6/6 [45:07<00:00, 451.24s/it]

tratamento_maxwell - LinearRegression: MAE = 0.0486, RMSE = 0.0805, RAE = 0.9043, RRSE = 1.2513
tratamento_maxwell - Ridge: MAE = 0.0346, RMSE = 0.0538, RAE = 0.6440, RRSE = 0.8356
tratamento_maxwell - RandomForest: MAE = 0.0426, RMSE = 0.0605, RAE = 0.7930, RRSE = 0.9406
Resultados salvos em 'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\MIMIC-25_resultados.xlsx'.


In [9]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.utils.validation import check_X_y
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np

# Funções de métricas

def calculate_rae(y_true, y_pred):
    return np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(y_true - np.mean(y_true)))

def calculate_rrse(y_true, y_pred):
    return np.sqrt(np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))

# Função para carregar datasets
def load_datasets(directory):
    datasets = {}
    if not os.path.exists(directory):
        print(f"Diretório '{directory}' não encontrado.")
        return datasets

    for file in os.listdir(directory):
        filepath = os.path.join(directory, file)
        filename, ext = os.path.splitext(file)

        if ext != '.txt':
            print(f"Ignorando arquivo não .txt: {file}")
            continue

        try:
            with open(filepath, 'r') as f:
                first_line = f.readline()
                delimiter = ',' if ',' in first_line else ('\t' if '\t' in first_line else ' ')
            datasets[filename] = pd.read_csv(filepath, delimiter=delimiter)
            print(f"{filename}: Delimitador detectado - '{delimiter}'")
        except Exception as e:
            print(f"Erro ao carregar {file}: {e}")

    print(f"{len(datasets)} arquivos carregados com sucesso de {directory}.")
    return datasets

# Função para processar um único modelo
def process_model(X_train, X_test, y_train, y_test, model_name, model, param_grid, repetitions):
    metrics = {'mae': [], 'rmse': [], 'rae': [], 'rrse': []}

    for _ in range(repetitions):
        grid = GridSearchCV(model, param_grid[model_name], cv=5, scoring='neg_mean_absolute_error', error_score='raise')

        try:
            grid.fit(X_train, y_train)
            best_model = grid.best_estimator_

            y_pred = best_model.predict(X_test)

            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            rae = calculate_rae(y_test, y_pred)
            rrse = calculate_rrse(y_test, y_pred)

            metrics['mae'].append(mae)
            metrics['rmse'].append(rmse)
            metrics['rae'].append(rae)
            metrics['rrse'].append(rrse)

        except Exception as e:
            print(f"{model_name}: Falha no treinamento - {e}")
            continue

    return {
        'mae': np.mean(metrics['mae']),
        'rmse': np.mean(metrics['rmse']),
        'rae': np.mean(metrics['rae']),
        'rrse': np.mean(metrics['rrse']),
        'best_params': grid.best_params_ if 'grid' in locals() else {}
    }

# Função para treinar e testar modelos
def test_regression_models(data_original, data_synthetic, output_directory, repetitions=30):
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(),
    }

    param_grid = {
        'LinearRegression': {},
        'Ridge': {'alpha': [0.1, 1, 10]},
        'RandomForest': {'n_estimators': [50, 100, 200]},
    }

    results = {}

    for name, df_original in tqdm(data_original.items(), desc="Processando datasets originais"):
        if name not in data_synthetic:
            print(f"{name}: Dados sintéticos correspondentes não encontrados.")
            continue

        df_synthetic = data_synthetic[name]

        df_original = df_original.dropna()
        df_synthetic = df_synthetic.dropna()

        if df_original.shape[0] < 2 or df_original.shape[1] < 2:
            print(f"{name}: Dataset original insuficiente. Tamanho: {df_original.shape}")
            continue

        if df_synthetic.shape[0] < 2 or df_synthetic.shape[1] < 2:
            print(f"{name}: Dataset sintético insuficiente. Tamanho: {df_synthetic.shape}")
            continue

        # Dados originais
        X_original = df_original.iloc[:, :-1]
        y_original = df_original.iloc[:, -1]

        # Dados combinados
        combined_df = pd.concat([df_original, df_synthetic], ignore_index=True)
        X_combined = combined_df.iloc[:, :-1]
        y_combined = combined_df.iloc[:, -1]

        # Validação dos dados
        try:
            X_original, y_original = check_X_y(X_original, y_original)
            X_combined, y_combined = check_X_y(X_combined, y_combined)
        except ValueError as e:
            print(f"{name}: Erro de validação dos dados - {e}")
            continue

        # Divisão de treino e teste
        X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X_original, y_original, test_size=0.2, random_state=42)
        X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

        # Treino e teste para cada configuração
        model_results_original = Parallel(n_jobs=-1)(
            delayed(process_model)(X_train_orig, X_test_orig, y_train_orig, y_test_orig, model_name, model, param_grid, repetitions)
            for model_name, model in models.items()
        )

        model_results_combined = Parallel(n_jobs=-1)(
            delayed(process_model)(X_train_comb, X_test_comb, y_train_comb, y_test_comb, model_name, model, param_grid, repetitions)
            for model_name, model in models.items()
        )

        for model_name, metrics_original, metrics_combined in zip(models.keys(), model_results_original, model_results_combined):
            results[f'{name}_{model_name}_original'] = metrics_original
            results[f'{name}_{model_name}_combined'] = metrics_combined

            print(f"{name} - {model_name} (Original): MAE = {metrics_original['mae']:.4f}, RMSE = {metrics_original['rmse']:.4f}")
            print(f"{name} - {model_name} (Combined): MAE = {metrics_combined['mae']:.4f}, RMSE = {metrics_combined['rmse']:.4f}")

    results_df = pd.DataFrame.from_dict(results, orient='index')
    output_path = os.path.join(output_directory, 'abordagem3-processamento_alisson.xlsx')

    try:
        os.makedirs(output_directory, exist_ok=True)
        results_df.to_excel(output_path)
        print(f"Resultados salvos em '{output_path}'.")
    except Exception as e:
        print(f"Erro ao salvar os resultados: {e}")

# Configuração dos diretórios
output_directory = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados'
data_original_dir = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\2-geracao-variaveis\MIMIC-50'
data_synthetic_dir = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\3-simulacao\MIMIC-50'

# Carregamento dos dados
data_original = load_datasets(data_original_dir)
data_synthetic = load_datasets(data_synthetic_dir)

if data_original and data_synthetic:
    test_regression_models(data_original, data_synthetic, output_directory, repetitions=30)


tratamento_china: Delimitador detectado - ','
Ignorando arquivo não .txt: tratamento_china_resultados_processamento_simples.xlsx
tratamento_cocomo81: Delimitador detectado - ','
Ignorando arquivo não .txt: tratamento_cocomo81_resultados_processamento_simples.xlsx
tratamento_desharnais: Delimitador detectado - ','
Ignorando arquivo não .txt: tratamento_desharnais_resultados_processamento_simples.xlsx
tratamento_maxwell: Delimitador detectado - ','
Ignorando arquivo não .txt: tratamento_maxwell_resultados_processamento_simples.xlsx
4 arquivos carregados com sucesso de C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\2-geracao-variaveis\MIMIC-50.
tratamento_china: Delimitador detectado - ','
Ignorando arquivo não .txt: tratamento_china.txt_resultados_alisson.xlsx
Ignorando arquivo não .txt: tratamento_china_resultados_processamento_simples.xlsx
tratamento_cocomo81: Delimitador detectado - ','
Ignorando arquivo

Processando datasets originais: 100%|██████████| 4/4 [00:00<00:00, 280.61it/s]

tratamento_china: Erro de validação dos dados - Input X contains NaN.
tratamento_cocomo81: Erro de validação dos dados - Input X contains NaN.
tratamento_desharnais: Erro de validação dos dados - Input X contains NaN.
tratamento_maxwell: Erro de validação dos dados - Input X contains NaN.
Resultados salvos em 'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\abordagem3-processamento_alisson.xlsx'.


# 4 - Software effort estimation modeling and fully connected artificial neural network optimization using soft computing techniques

Divisão: 70 treino e 30 teste

In [23]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils.validation import check_X_y
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np

# Função para calcular RAE
def calculate_rae(y_true, y_pred):
    return np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(y_true - np.mean(y_true)))

# Função para calcular RRSE
def calculate_rrse(y_true, y_pred):
    return np.sqrt(np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))

# Função para calcular VAF (Variance Accounted For)
def calculate_vaf(y_true, y_pred):
    return 100 * (1 - np.var(y_true - y_pred) / np.var(y_true))

# Função para calcular ED (Euclidean Distance)
def calculate_ed(y_true, y_pred):
    return np.sqrt(np.sum((y_true - y_pred) ** 2))

# Função para calcular MdMRE (Median Magnitude of Relative Error)
def calculate_mdmre(y_true, y_pred):
    return np.median(np.abs((y_true - y_pred) / y_true))

# Função para calcular MD (Mean Deviation)
def calculate_md(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

# Função para calcular SA (Standard Accuracy)
def calculate_sa(y_true, y_pred):
    return 100 * (1 - np.mean(np.abs((y_true - y_pred) / y_true)))

# Função para carregar datasets
def load_datasets(directory):
    datasets = {}
    if not os.path.exists(directory):
        print(f"Diretório '{directory}' não encontrado.")
        return datasets

    for file in os.listdir(directory):
        filepath = os.path.join(directory, file)
        filename, ext = os.path.splitext(file)

        try:
            if ext == '.txt':
                with open(filepath, 'r') as f:
                    first_line = f.readline()
                    delimiter = ',' if ',' in first_line else ('\t' if '\t' in first_line else ' ')
                datasets[filename] = pd.read_csv(filepath, delimiter=delimiter)
                print(f"{filename}: Delimitador detectado - '{delimiter}'")
            else:
                print(f"Formato não suportado: {file}")
        except Exception as e:
            print(f"Erro ao carregar {file}: {e}")

    print(f"{len(datasets)} arquivos carregados com sucesso de {directory}.")
    return datasets

# Função para processar um único modelo
def process_model(X_train, X_test, y_train, y_test, model_name, model, param_grid, repetitions):
    metrics = {
        'mae': [], 'rmse': [], 'rae': [], 'rrse': [], 'vaf': [], 'ed': [], 
        'mdmre': [], 'r2': [], 'md': [], 'sa': []
    }

    for _ in range(repetitions):
        grid = GridSearchCV(model, param_grid[model_name], cv=5, scoring='neg_mean_absolute_error', error_score='raise')

        try:
            grid.fit(X_train, y_train)
            best_model = grid.best_estimator_

            # Previsões no conjunto de teste
            y_pred = best_model.predict(X_test)

            # Cálculo das métricas
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            rae = calculate_rae(y_test, y_pred)
            rrse = calculate_rrse(y_test, y_pred)
            vaf = calculate_vaf(y_test, y_pred)
            ed = calculate_ed(y_test, y_pred)
            mdmre = calculate_mdmre(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            md = calculate_md(y_test, y_pred)
            sa = calculate_sa(y_test, y_pred)

            metrics['mae'].append(mae)
            metrics['rmse'].append(rmse)
            metrics['rae'].append(rae)
            metrics['rrse'].append(rrse)
            metrics['vaf'].append(vaf)
            metrics['ed'].append(ed)
            metrics['mdmre'].append(mdmre)
            metrics['r2'].append(r2)
            metrics['md'].append(md)
            metrics['sa'].append(sa)

        except Exception as e:
            print(f"{model_name}: Falha no treinamento - {e}")
            continue

    # Calcular médias das métricas
    return {
        'mae': np.mean(metrics['mae']),
        'rmse': np.mean(metrics['rmse']),
        'rae': np.mean(metrics['rae']),
        'rrse': np.mean(metrics['rrse']),
        'vaf': np.mean(metrics['vaf']),
        'ed': np.mean(metrics['ed']),
        'mdmre': np.mean(metrics['mdmre']),
        'r2': np.mean(metrics['r2']),
        'md': np.mean(metrics['md']),
        'sa': np.mean(metrics['sa']),
        'best_params': grid.best_params_ if 'grid' in locals() else {}
    }

# Função para treinar e testar modelos de regressão
def test_regression_models(data, directory, output_directory, repetitions=30):
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(),
    }

    param_grid = {
        'LinearRegression': {},
        'Ridge': {'alpha': [0.1, 1, 10]},
        'RandomForest': {'n_estimators': [50, 100, 200]},
    }

    results = {}

    for name, df in tqdm(data.items(), desc="Processando datasets"):
        df = df.dropna()

        if df.shape[0] < 2 or df.shape[1] < 2:
            print(f"{name}: Dataset insuficiente para treino. Tamanho: {df.shape}")
            continue

        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]

        try:
            X, y = check_X_y(X, y)
        except ValueError as e:
            print(f"{name}: Erro de validação dos dados - {e}")
            continue

        # Divisão dos dados em treinamento (70%) e teste (30%)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Processar cada modelo em paralelo
        model_results = Parallel(n_jobs=-1)(
            delayed(process_model)(X_train, X_test, y_train, y_test, model_name, model, param_grid, repetitions)
            for model_name, model in models.items()
        )

        # Armazenar resultados
        for model_name, metrics in zip(models.keys(), model_results):
            results[f'{name}_{model_name}'] = metrics
            print(f"{name} - {model_name}: MAE = {metrics['mae']:.4f}, RMSE = {metrics['rmse']:.4f}, "
                  f"RAE = {metrics['rae']:.4f}, RRSE = {metrics['rrse']:.4f}, VAF = {metrics['vaf']:.4f}, "
                  f"ED = {metrics['ed']:.4f}, MdMRE = {metrics['mdmre']:.4f}, R2 = {metrics['r2']:.4f}, "
                  f"MD = {metrics['md']:.4f}, SA = {metrics['sa']:.4f}")

    results_df = pd.DataFrame.from_dict(results, orient='index')
    output_path = os.path.join(output_directory, f'{os.path.basename(directory)}_resultados.xlsx')

    try:
        os.makedirs(output_directory, exist_ok=True)
        results_df.to_excel(output_path)
        print(f"Resultados salvos em '{output_path}'.")
    except Exception as e:
        print(f"Erro ao salvar os resultados: {e}")

# Processamento de múltiplos diretórios
output_directory = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\artigo4'
directories = [
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\1-tratamento',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\2-geracao\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\2-geracao-variaveis\MIMIC-25',
]

for directory in tqdm(directories, desc="Processando diretórios"):
    data = load_datasets(directory)
    if data:
        test_regression_models(data, directory, output_directory, repetitions=30)

Processando diretórios:   0%|          | 0/6 [00:00<?, ?it/s]

Formato não suportado: resultado_consolidado_tratamento.xlsx
tratamento_china: Delimitador detectado - ','
Formato não suportado: tratamento_china_resultados_kfold.xlsx
tratamento_cocomo81: Delimitador detectado - ','
Formato não suportado: tratamento_cocomo81_resultados_kfold.xlsx
tratamento_desharnais: Delimitador detectado - ','
Formato não suportado: tratamento_desharnais_resultados_kfold.xlsx
tratamento_maxwell: Delimitador detectado - ','
Formato não suportado: tratamento_maxwell_resultados_kfold.xlsx
4 arquivos carregados com sucesso de C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado.


tratamento_china - LinearRegression: MAE = 0.0071, RMSE = 0.0193, RAE = 0.1285, RRSE = 0.1878, VAF = 96.4804, ED = 0.2364, MdMRE = 0.1163, R2 = 0.9647, MD = 0.0071, SA = 66.9181
tratamento_china - Ridge: MAE = 0.0071, RMSE = 0.0165, RAE = 0.1279, RRSE = 0.1602, VAF = 97.4793, ED = 0.2016, MdMRE = 0.1020, R2 = 0.9743, MD = 0.0071, SA = 72.1575
tratamento_china - RandomForest: MAE = 0.0067, RMSE = 0.0247, RAE = 0.1213, RRSE = 0.2399, VAF = 94.3278, ED = 0.3020, MdMRE = 0.0804, R2 = 0.9424, MD = 0.0067, SA = 90.0225


tratamento_cocomo81 - LinearRegression: MAE = 0.0838, RMSE = 0.1862, RAE = 0.7670, RRSE = 0.8391, VAF = 37.7891, ED = 0.8116, MdMRE = 2.5408, R2 = 0.2960, MD = 0.0838, SA = -52413.8265
tratamento_cocomo81 - Ridge: MAE = 0.0719, RMSE = 0.1907, RAE = 0.6586, RRSE = 0.8596, VAF = 29.9683, ED = 0.8314, MdMRE = 1.9430, R2 = 0.2611, MD = 0.0719, SA = -33221.9616
tratamento_cocomo81 - RandomForest: MAE = 0.0536, RMSE = 0.1517, RAE = 0.4908, RRSE = 0.6836, VAF = 55.6544, ED = 0.6612, MdMRE = 1.1556, R2 = 0.5323, MD = 0.0536, SA = -712.7909


tratamento_desharnais - LinearRegression: MAE = 0.1066, RMSE = 0.1326, RAE = 0.9025, RRSE = 0.8545, VAF = 31.6716, ED = 0.6632, MdMRE = 0.6158, R2 = 0.2698, MD = 0.1066, SA = -inf
tratamento_desharnais - Ridge: MAE = 0.0852, RMSE = 0.1084, RAE = 0.7215, RRSE = 0.6985, VAF = 53.8963, ED = 0.5421, MdMRE = 0.5059, R2 = 0.5121, MD = 0.0852, SA = -inf
tratamento_desharnais - RandomForest: MAE = 0.0840, RMSE = 0.1102, RAE = 0.7114, RRSE = 0.7100, VAF = 49.5980, ED = 0.5510, MdMRE = 0.4498, R2 = 0.4949, MD = 0.0840, SA = -inf


...

tratamento_china - LinearRegression: MAE = 0.0010, RMSE = 0.0014, RAE = 0.0182, RRSE = 0.0136, VAF = 99.9816, ED = 0.0171, MdMRE = 0.0265, R2 = 0.9998, MD = 0.0010, SA = 90.4306
tratamento_china - Ridge: MAE = 0.0034, RMSE = 0.0066, RAE = 0.0609, RRSE = 0.0647, VAF = 99.5857, ED = 0.0814, MdMRE = 0.0604, R2 = 0.9958, MD = 0.0034, SA = 83.6306
tratamento_china - RandomForest: MAE = 0.0033, RMSE = 0.0176, RAE = 0.0594, RRSE = 0.1710, VAF = 97.0982, ED = 0.2153, MdMRE = 0.0249, R2 = 0.9707, MD = 0.0033, SA = 96.0820


tratamento_cocomo81 - LinearRegression: MAE = 0.0748, RMSE = 0.1754, RAE = 0.6852, RRSE = 0.7902, VAF = 40.7465, ED = 0.7643, MdMRE = 2.5379, R2 = 0.3755, MD = 0.0748, SA = -25791.8950
tratamento_cocomo81 - Ridge: MAE = 0.0675, RMSE = 0.1696, RAE = 0.6182, RRSE = 0.7645, VAF = 43.7140, ED = 0.7394, MdMRE = 2.4244, R2 = 0.4156, MD = 0.0675, SA = -23014.8546
tratamento_cocomo81 - RandomForest: MAE = 0.0465, RMSE = 0.1378, RAE = 0.4259, RRSE = 0.6210, VAF = 63.6507, ED = 0.6006, MdMRE = 0.6968, R2 = 0.6136, MD = 0.0465, SA = -514.9988


tratamento_desharnais - LinearRegression: MAE = 0.0854, RMSE = 0.1198, RAE = 0.7232, RRSE = 0.7716, VAF = 41.1911, ED = 0.5989, MdMRE = 0.3829, R2 = 0.4046, MD = 0.0854, SA = -inf
tratamento_desharnais - Ridge: MAE = 0.0835, RMSE = 0.1182, RAE = 0.7071, RRSE = 0.7616, VAF = 42.7859, ED = 0.5911, MdMRE = 0.4351, R2 = 0.4200, MD = 0.0835, SA = -inf
tratamento_desharnais - RandomForest: MAE = 0.0863, RMSE = 0.1177, RAE = 0.7311, RRSE = 0.7581, VAF = 42.7784, ED = 0.5884, MdMRE = 0.3960, R2 = 0.4249, MD = 0.0863, SA = -inf


Processando diretórios: 100%|██████████| 6/6 [23:58<00:00, 239.74s/it]

tratamento_maxwell - LinearRegression: MAE = 0.0589, RMSE = 0.0864, RAE = 0.8089, RRSE = 0.8933, VAF = 28.2444, ED = 0.3765, MdMRE = 0.4852, R2 = 0.2020, MD = 0.0589, SA = -inf
tratamento_maxwell - Ridge: MAE = 0.0296, RMSE = 0.0441, RAE = 0.4062, RRSE = 0.4561, VAF = 81.5303, ED = 0.1923, MdMRE = 0.4120, R2 = 0.7919, MD = 0.0296, SA = -inf
tratamento_maxwell - RandomForest: MAE = 0.0401, RMSE = 0.0559, RAE = 0.5515, RRSE = 0.5777, VAF = 67.0111, ED = 0.2435, MdMRE = 0.4701, R2 = 0.6656, MD = 0.0401, SA = -inf
Resultados salvos em 'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\artigo4\MIMIC-25_resultados.xlsx'.


# 5 - Recommendation of Machine Learning Techniques for Software Effort Estimation using Multi-Criteria Decision Making

Divisão: 70-30

In [24]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.utils.validation import check_X_y
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np
from scipy.stats import pearsonr

# Função para calcular MMRE (Mean Magnitude of Relative Error)
def calculate_mmre(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

# Função para calcular MDARE (Median of Absolute Residual Error)
def calculate_mdare(y_true, y_pred):
    return np.median(np.abs(y_true - y_pred))

# Função para calcular MBRE (Mean Balanced Relative Error)
def calculate_mbre(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred) / np.minimum(y_true, y_pred))

# Função para calcular PRED (0.25)
def calculate_pred(y_true, y_pred, threshold=0.25):
    relative_errors = np.abs((y_true - y_pred) / y_true)
    return np.mean(relative_errors <= threshold)

# Função para carregar datasets
def load_datasets(directory):
    datasets = {}
    if not os.path.exists(directory):
        print(f"Diretório '{directory}' não encontrado.")
        return datasets

    for file in os.listdir(directory):
        filepath = os.path.join(directory, file)
        filename, ext = os.path.splitext(file)

        try:
            if ext == '.txt':
                with open(filepath, 'r') as f:
                    first_line = f.readline()
                    delimiter = ',' if ',' in first_line else ('\t' if '\t' in first_line else ' ')
                datasets[filename] = pd.read_csv(filepath, delimiter=delimiter)
                print(f"{filename}: Delimitador detectado - '{delimiter}'")
            else:
                print(f"Formato não suportado: {file}")
        except Exception as e:
            print(f"Erro ao carregar {file}: {e}")

    print(f"{len(datasets)} arquivos carregados com sucesso de {directory}.")
    return datasets

# Função para processar um único modelo
def process_model(X_train, X_test, y_train, y_test, model_name, model, param_grid, repetitions):
    metrics = {
        'rmse': [], 'mmre': [], 'mdare': [], 'mbre': [], 
        'pearson_r': [], 'pred_025': []
    }

    for _ in range(repetitions):
        grid = GridSearchCV(model, param_grid[model_name], cv=5, scoring='neg_mean_absolute_error', error_score='raise')

        try:
            grid.fit(X_train, y_train)
            best_model = grid.best_estimator_

            # Previsões no conjunto de teste
            y_pred = best_model.predict(X_test)

            # Cálculo das métricas
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mmre = calculate_mmre(y_test, y_pred)
            mdare = calculate_mdare(y_test, y_pred)
            mbre = calculate_mbre(y_test, y_pred)
            pearson_r, _ = pearsonr(y_test, y_pred)
            pred_025 = calculate_pred(y_test, y_pred)

            metrics['rmse'].append(rmse)
            metrics['mmre'].append(mmre)
            metrics['mdare'].append(mdare)
            metrics['mbre'].append(mbre)
            metrics['pearson_r'].append(pearson_r)
            metrics['pred_025'].append(pred_025)

        except Exception as e:
            print(f"{model_name}: Falha no treinamento - {e}")
            continue

    # Calcular médias das métricas
    return {
        'rmse': np.mean(metrics['rmse']),
        'mmre': np.mean(metrics['mmre']),
        'mdare': np.mean(metrics['mdare']),
        'mbre': np.mean(metrics['mbre']),
        'pearson_r': np.mean(metrics['pearson_r']),
        'pred_025': np.mean(metrics['pred_025']),
        'best_params': grid.best_params_ if 'grid' in locals() else {}
    }

# Função para treinar e testar modelos de regressão
def test_regression_models(data, directory, output_directory, repetitions=30):
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(),
    }

    param_grid = {
        'LinearRegression': {},
        'Ridge': {'alpha': [0.1, 1, 10]},
        'RandomForest': {'n_estimators': [50, 100, 200]},
    }

    results = {}

    for name, df in tqdm(data.items(), desc="Processando datasets"):
        df = df.dropna()

        if df.shape[0] < 2 or df.shape[1] < 2:
            print(f"{name}: Dataset insuficiente para treino. Tamanho: {df.shape}")
            continue

        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]

        try:
            X, y = check_X_y(X, y)
        except ValueError as e:
            print(f"{name}: Erro de validação dos dados - {e}")
            continue

        # Divisão dos dados em treinamento (80%) e teste (20%)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Processar cada modelo em paralelo
        model_results = Parallel(n_jobs=-1)(
            delayed(process_model)(X_train, X_test, y_train, y_test, model_name, model, param_grid, repetitions)
            for model_name, model in models.items()
        )

        # Armazenar resultados
        for model_name, metrics in zip(models.keys(), model_results):
            results[f'{name}_{model_name}'] = metrics
            print(f"{name} - {model_name}: RMSE = {metrics['rmse']:.4f}, MMRE = {metrics['mmre']:.4f}, "
                  f"MDARE = {metrics['mdare']:.4f}, MBRE = {metrics['mbre']:.4f}, "
                  f"Pearson r = {metrics['pearson_r']:.4f}, PRED(0.25) = {metrics['pred_025']:.4f}")

    results_df = pd.DataFrame.from_dict(results, orient='index')
    output_path = os.path.join(output_directory, f'{os.path.basename(directory)}_resultados.xlsx')

    try:
        os.makedirs(output_directory, exist_ok=True)
        results_df.to_excel(output_path)
        print(f"Resultados salvos em '{output_path}'.")
    except Exception as e:
        print(f"Erro ao salvar os resultados: {e}")

# Processamento de múltiplos diretórios
output_directory = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\artigo5'
directories = [
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\1-tratamento',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\correlacao\2-geracao\MIMIC-25',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\1-tratamento-target-encoding-normalizado',
    r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\50\2-geracao-variaveis\MIMIC-25',
]

for directory in tqdm(directories, desc="Processando diretórios"):
    data = load_datasets(directory)
    if data:
        test_regression_models(data, directory, output_directory, repetitions=30)

Processando diretórios:   0%|          | 0/6 [00:00<?, ?it/s]

Formato não suportado: resultado_consolidado_tratamento.xlsx
tratamento_china: Delimitador detectado - ','
Formato não suportado: tratamento_china_resultados_kfold.xlsx
tratamento_cocomo81: Delimitador detectado - ','
Formato não suportado: tratamento_cocomo81_resultados_kfold.xlsx
tratamento_desharnais: Delimitador detectado - ','
Formato não suportado: tratamento_desharnais_resultados_kfold.xlsx
tratamento_maxwell: Delimitador detectado - ','
Formato não suportado: tratamento_maxwell_resultados_kfold.xlsx
4 arquivos carregados com sucesso de C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado.


tratamento_china - LinearRegression: RMSE = 0.0233, MMRE = 0.4073, MDARE = 0.0038, MBRE = 0.2878, Pearson r = 0.9736, PRED(0.25) = 0.7100
tratamento_china - Ridge: RMSE = 0.0158, MMRE = 0.2881, MDARE = 0.0024, MBRE = 0.5017, Pearson r = 0.9944, PRED(0.25) = 0.7800
tratamento_china - RandomForest: RMSE = 0.0288, MMRE = 0.1114, MDARE = 0.0018, MBRE = 0.1182, Pearson r = 0.9840, PRED(0.25) = 0.9687


tratamento_cocomo81 - LinearRegression: RMSE = 0.1587, MMRE = 1634.3269, MDARE = 0.1100, MBRE = 10.8253, Pearson r = 0.4177, PRED(0.25) = 0.0769
tratamento_cocomo81 - Ridge: RMSE = 0.0608, MMRE = 1044.0256, MDARE = 0.0553, MBRE = 23.3034, Pearson r = 0.6967, PRED(0.25) = 0.2308
tratamento_cocomo81 - RandomForest: RMSE = 0.0426, MMRE = 9.2042, MDARE = 0.0080, MBRE = 9.5483, Pearson r = 0.5580, PRED(0.25) = 0.1154


tratamento_desharnais - LinearRegression: RMSE = 0.1102, MMRE = inf, MDARE = 0.0633, MBRE = -0.0825, Pearson r = 0.7754, PRED(0.25) = 0.2353
tratamento_desharnais - Ridge: RMSE = 0.1019, MMRE = inf, MDARE = 0.0642, MBRE = -5.2666, Pearson r = 0.7783, PRED(0.25) = 0.2941
tratamento_desharnais - RandomForest: RMSE = 0.0911, MMRE = inf, MDARE = 0.0590, MBRE = inf, Pearson r = 0.8176, PRED(0.25) = 0.2333


...

tratamento_china - LinearRegression: RMSE = 0.0013, MMRE = 0.1008, MDARE = 0.0008, MBRE = 0.1026, Pearson r = 0.9999, PRED(0.25) = 0.8900
tratamento_china - Ridge: RMSE = 0.0059, MMRE = 0.1545, MDARE = 0.0016, MBRE = 0.1554, Pearson r = 0.9995, PRED(0.25) = 0.8300
tratamento_china - RandomForest: RMSE = 0.0208, MMRE = 0.0441, MDARE = 0.0007, MBRE = 0.0473, Pearson r = 0.9900, PRED(0.25) = 0.9963


tratamento_cocomo81 - LinearRegression: RMSE = 0.0830, MMRE = 786.7234, MDARE = 0.0607, MBRE = 31.3211, Pearson r = 0.8288, PRED(0.25) = 0.0769
tratamento_cocomo81 - Ridge: RMSE = 0.0616, MMRE = 657.1756, MDARE = 0.0574, MBRE = 24.8270, Pearson r = 0.8198, PRED(0.25) = 0.0000
tratamento_cocomo81 - RandomForest: RMSE = 0.0324, MMRE = 8.2552, MDARE = 0.0072, MBRE = 8.5091, Pearson r = 0.8685, PRED(0.25) = 0.1410


tratamento_desharnais - LinearRegression: RMSE = 0.1091, MMRE = inf, MDARE = 0.0939, MBRE = inf, Pearson r = 0.7191, PRED(0.25) = 0.3529
tratamento_desharnais - Ridge: RMSE = 0.1084, MMRE = inf, MDARE = 0.0858, MBRE = inf, Pearson r = 0.7249, PRED(0.25) = 0.3529
tratamento_desharnais - RandomForest: RMSE = 0.0933, MMRE = inf, MDARE = 0.0603, MBRE = inf, Pearson r = 0.8058, PRED(0.25) = 0.2784


Processando diretórios: 100%|██████████| 6/6 [36:33<00:00, 365.55s/it]

tratamento_maxwell - LinearRegression: RMSE = 0.0805, MMRE = 1.3582, MDARE = 0.0316, MBRE = 0.7350, Pearson r = 0.7336, PRED(0.25) = 0.1538
tratamento_maxwell - Ridge: RMSE = 0.0538, MMRE = 1.0627, MDARE = 0.0218, MBRE = 0.6345, Pearson r = 0.8653, PRED(0.25) = 0.4615
tratamento_maxwell - RandomForest: RMSE = 0.0602, MMRE = 1.3840, MDARE = 0.0289, MBRE = 1.4627, Pearson r = 0.7578, PRED(0.25) = 0.3026
Resultados salvos em 'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\resultados\artigo5\MIMIC-25_resultados.xlsx'.
